In [1]:
# import modules
import json
import pandas as pd
import re
import regex
from datetime import datetime
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)

import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
import unicodedata

%reload_ext autoreload
%autoreload 2
from contractions import *

# nlp = spacy.load('en_core', parse=True, tag=True, entity=True)
# #nlp_vec = spacy.load('en_vecs', parse = True, tag=True, #entity=True)
# tokenizer = ToktokTokenizer()
# stopword_list = nltk.corpus.stopwords.words('english')
# stopword_list.remove('no')
# stopword_list.remove('not')

In [2]:
# day_count = (end_date - start_date).days + 1
# for single_date in (start_date + timedelta(n) for n in range(day_count)):
#     print strftime("%Y-%m-&d", single_date.timetuple()) 
    
#     filenames = '2016-07-' + str()

In [3]:
tw = pd.read_json('data/2020-03-20.json')
tw

,id,screen_name,user_id,time,link,text,source
0,1241177108442947584,brianschatz,47747074,2020-03-20T21:37:37-04:00,https://www.twitter.com/brianschatz/statuses/1...,@JNTHN_LCKWD @jbarro @CoryBooker Understood. W...,Twitter for iPhone
1,1241048506468622336,PattyMurray,293131808,2020-03-20T13:06:36-04:00,https://www.twitter.com/PattyMurray/statuses/1...,In accordance with current public health recom...,Twitter Web App
2,1241139330392961024,amyklobuchar,33537967,2020-03-20T19:07:30-04:00,https://www.twitter.com/amyklobuchar/statuses/...,The American people should know if they're sic...,Twitter Web App
3,1241044145994715136,dccc,14676022,2020-03-20T12:49:16-04:00,https://www.twitter.com/Court_Rice/statuses/12...,RT @Court_Rice LEADERSHIP: @CartwrightPA launc...,Twitter Web App
4,1241157748408176640,garretgraves,587100576,2020-03-20T20:20:41-04:00,https://www.twitter.com/garretgraves/statuses/...,https://www.wbrz.com/news/wbrz-business-direct...,Twitter for iPhone
...,...,...,...,...,...,...,...
4684,1241065460478676992,JGO_2020,3857977018,2020-03-20T14:13:58-04:00,https://www.twitter.com/Jenniffer2012/statuses...,RT @Jenniffer2012 Esto hará disponible más rec...,Twitter for iPhone
4685,1241065450966077440,JGO_2020,3857977018,2020-03-20T14:13:56-04:00,https://www.twitter.com/Jenniffer2012/statuses...,RT @Jenniffer2012 Envié una carta a la Adminis...,Twitter for iPhone
4686,1241057746050580480,JGO_2020,3857977018,2020-03-20T13:43:19-04:00,https://www.twitter.com/RepJenniffer/statuses/...,RT @RepJenniffer Sent a letter to @SBAgov Admi...,Twitter for iPhone
4687,1241053140973166592,JGO_2020,3857977018,2020-03-20T13:25:01-04:00,https://www.twitter.com/Jenniffer2012/statuses...,RT @Jenniffer2012 Excelente noticia del tesoro...,Twitter for iPhone


In [4]:
df = tw.drop(columns=["link", "source", "user_id"], axis=1)

In [5]:
df.head()

,id,screen_name,time,text
0,1241177108442947584,brianschatz,2020-03-20T21:37:37-04:00,@JNTHN_LCKWD @jbarro @CoryBooker Understood. W...
1,1241048506468622336,PattyMurray,2020-03-20T13:06:36-04:00,In accordance with current public health recom...
2,1241139330392961024,amyklobuchar,2020-03-20T19:07:30-04:00,The American people should know if they're sic...
3,1241044145994715136,dccc,2020-03-20T12:49:16-04:00,RT @Court_Rice LEADERSHIP: @CartwrightPA launc...
4,1241157748408176640,garretgraves,2020-03-20T20:20:41-04:00,https://www.wbrz.com/news/wbrz-business-direct...


In [6]:
format = "%Y-%m-%dT%H:%M:%S%z"
df.time = df.time.apply(lambda x: datetime.strptime(x, format))
df

,id,screen_name,time,text
0,1241177108442947584,brianschatz,2020-03-20 21:37:37-04:00,@JNTHN_LCKWD @jbarro @CoryBooker Understood. W...
1,1241048506468622336,PattyMurray,2020-03-20 13:06:36-04:00,In accordance with current public health recom...
2,1241139330392961024,amyklobuchar,2020-03-20 19:07:30-04:00,The American people should know if they're sic...
3,1241044145994715136,dccc,2020-03-20 12:49:16-04:00,RT @Court_Rice LEADERSHIP: @CartwrightPA launc...
4,1241157748408176640,garretgraves,2020-03-20 20:20:41-04:00,https://www.wbrz.com/news/wbrz-business-direct...
...,...,...,...,...
4684,1241065460478676992,JGO_2020,2020-03-20 14:13:58-04:00,RT @Jenniffer2012 Esto hará disponible más rec...
4685,1241065450966077440,JGO_2020,2020-03-20 14:13:56-04:00,RT @Jenniffer2012 Envié una carta a la Adminis...
4686,1241057746050580480,JGO_2020,2020-03-20 13:43:19-04:00,RT @RepJenniffer Sent a letter to @SBAgov Admi...
4687,1241053140973166592,JGO_2020,2020-03-20 13:25:01-04:00,RT @Jenniffer2012 Excelente noticia del tesoro...


In [7]:
df2 = df.text
df2[0]

'@JNTHN_LCKWD @jbarro @CoryBooker Understood. Working this hard. Don’t know what the result will be.'

In [8]:
df2[6]

'RT @RepFletcher When it comes to combating #COVID19, Texans should not have to worry about their insurance status. Today, I joined  @RepLloydDoggett and our Texas colleagues to call on Governor Abbot to expand coverage for #COVID19 care for Texans. http://pbs.twimg.com/media/ETmLzjqXYAAfhDV.png http://pbs.twimg.com/media/ETmLz4BXkAA0faZ.png'

In [9]:
df2[2]

'The American people should know if they\'re sick or not. \n\nThat’s the only way we can know if people developed immunities—then we will know which workers can be back on the front lines, who can give plasma to other people, &amp; much more. \n\nWe need to get more testing for the people. https://twitter.com/esmemurphy/status/1240752676691947527 QT @esmemurphy .@amyklobuchar says she finds lack of access to testing "appalling" says her office is getting large numbers of constituent calls needing help with Covid-19 concerns @wcco 5 and 6 pm'

In [10]:
df2[7]

'@bubbadog1 And hockey!'

## Count numbere of words

In [11]:
def num_of_words(df, col):
    df['word_count'] = df[col].apply(lambda x: len(str(x).split(" ")))
    print(df[[col, 'word_count']])

In [12]:
num_of_words(df, 'text')

                                                   text  word_count
0     @JNTHN_LCKWD @jbarro @CoryBooker Understood. W...          14
1     In accordance with current public health recom...          40
2     The American people should know if they're sic...          85
3     RT @Court_Rice LEADERSHIP: @CartwrightPA launc...          29
4     https://www.wbrz.com/news/wbrz-business-direct...           1
...                                                 ...         ...
4684  RT @Jenniffer2012 Esto hará disponible más rec...          28
4685  RT @Jenniffer2012 Envié una carta a la Adminis...          73
4686  RT @RepJenniffer Sent a letter to @SBAgov Admi...          43
4687  RT @Jenniffer2012 Excelente noticia del tesoro...          58
4688  RT @Jenniffer2012 Esta mañana tuve un conferen...          42

[4689 rows x 2 columns]


## Count number of characters

In [13]:
def num_of_chars(df, col):
    df['char_count'] = df[col].str.len()
    print(df[[col, 'char_count']])

In [14]:
num_of_chars(df, 'text')

                                                   text  char_count
0     @JNTHN_LCKWD @jbarro @CoryBooker Understood. W...          99
1     In accordance with current public health recom...         270
2     The American people should know if they're sic...         540
3     RT @Court_Rice LEADERSHIP: @CartwrightPA launc...         298
4     https://www.wbrz.com/news/wbrz-business-direct...          76
...                                                 ...         ...
4684  RT @Jenniffer2012 Esto hará disponible más rec...         185
4685  RT @Jenniffer2012 Envié una carta a la Adminis...         604
4686  RT @RepJenniffer Sent a letter to @SBAgov Admi...         343
4687  RT @Jenniffer2012 Excelente noticia del tesoro...         414
4688  RT @Jenniffer2012 Esta mañana tuve un conferen...         289

[4689 rows x 2 columns]


## Average World Length

In [15]:
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

def avg_word_length(df, col):
    df['avg_wrd'] = df[col].apply(lambda x: avg_word(x))
    print(df[[col, 'avg_wrd']].head())

In [16]:
avg_word_length(df, 'text')

                                                text    avg_wrd
0  @JNTHN_LCKWD @jbarro @CoryBooker Understood. W...   6.142857
1  In accordance with current public health recom...   5.775000
2  The American people should know if they're sic...   5.317647
3  RT @Court_Rice LEADERSHIP: @CartwrightPA launc...   9.241379
4  https://www.wbrz.com/news/wbrz-business-direct...  76.000000


In [17]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [18]:
def stop_words(df, col):
    df['stopwords'] = df[col].apply(lambda x: len([x for x in x.split() if x in stop]))
    print(df[[col, 'stopwords']].head())

In [19]:
stop_words(df, 'text')

                                                text  stopwords
0  @JNTHN_LCKWD @jbarro @CoryBooker Understood. W...          4
1  In accordance with current public health recom...         16
2  The American people should know if they're sic...         31
3  RT @Court_Rice LEADERSHIP: @CartwrightPA launc...         11
4  https://www.wbrz.com/news/wbrz-business-direct...          0


In [20]:
def hash_tags(df, col):
    df['hashtags'] = df[col].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
    print(df[[col, 'hashtags']].head())

In [21]:
hash_tags(df, 'text')

                                                text  hashtags
0  @JNTHN_LCKWD @jbarro @CoryBooker Understood. W...         0
1  In accordance with current public health recom...         0
2  The American people should know if they're sic...         0
3  RT @Court_Rice LEADERSHIP: @CartwrightPA launc...         1
4  https://www.wbrz.com/news/wbrz-business-direct...         0


In [22]:
df.hashtags.value_counts()

0    2787
1    1187
2     477
3     158
4      53
5      16
6       7
7       4
Name: hashtags, dtype: int64

In [23]:
def num_numerics(df, col):
    df['numerics'] = df[col].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
    print(df[[col, 'numerics']].head())

num_numerics(df, 'text')

                                                text  numerics
0  @JNTHN_LCKWD @jbarro @CoryBooker Understood. W...         0
1  In accordance with current public health recom...         0
2  The American people should know if they're sic...         2
3  RT @Court_Rice LEADERSHIP: @CartwrightPA launc...         0
4  https://www.wbrz.com/news/wbrz-business-direct...         0


In [24]:
df.numerics.value_counts()

0    3932
1     584
2     147
3      17
4       8
7       1
Name: numerics, dtype: int64

In [25]:
def num_callouts(df, col):
    df['callouts'] = df[col].apply(lambda x: len([x for x in x.split() if x.startswith('@')]))
    print(df[[col, 'callouts']].head())

num_callouts(df, 'text')

                                                text  callouts
0  @JNTHN_LCKWD @jbarro @CoryBooker Understood. W...         3
1  In accordance with current public health recom...         0
2  The American people should know if they're sic...         2
3  RT @Court_Rice LEADERSHIP: @CartwrightPA launc...         2
4  https://www.wbrz.com/news/wbrz-business-direct...         0


In [26]:
df.callouts.value_counts()

1     1769
0     1560
2      760
3      309
4      136
5       64
6       43
7       10
9        9
8        6
13       5
10       5
11       5
15       3
14       2
16       1
12       1
19       1
Name: callouts, dtype: int64

In [27]:
df.head()

,id,screen_name,time,text,word_count,char_count,avg_wrd,stopwords,hashtags,numerics,callouts
0,1241177108442947584,brianschatz,2020-03-20 21:37:37-04:00,@JNTHN_LCKWD @jbarro @CoryBooker Understood. W...,14,99,6.142857,4,0,0,3
1,1241048506468622336,PattyMurray,2020-03-20 13:06:36-04:00,In accordance with current public health recom...,40,270,5.775000,16,0,0,0
2,1241139330392961024,amyklobuchar,2020-03-20 19:07:30-04:00,The American people should know if they're sic...,85,540,5.317647,31,0,2,2
3,1241044145994715136,dccc,2020-03-20 12:49:16-04:00,RT @Court_Rice LEADERSHIP: @CartwrightPA launc...,29,298,9.241379,11,1,0,2
4,1241157748408176640,garretgraves,2020-03-20 20:20:41-04:00,https://www.wbrz.com/news/wbrz-business-direct...,1,76,76.000000,0,0,0,0


# Basic Text Processing

In [28]:
X = df2
X.head()

0    @JNTHN_LCKWD @jbarro @CoryBooker Understood. W...
1    In accordance with current public health recom...
2    The American people should know if they're sic...
3    RT @Court_Rice LEADERSHIP: @CartwrightPA launc...
4    https://www.wbrz.com/news/wbrz-business-direct...
Name: text, dtype: object

In [29]:
tw = pd.DataFrame(df2, columns=["text"])
tw.head()

,text
0,@JNTHN_LCKWD @jbarro @CoryBooker Understood. W...
1,In accordance with current public health recom...
2,The American people should know if they're sic...
3,RT @Court_Rice LEADERSHIP: @CartwrightPA launc...
4,https://www.wbrz.com/news/wbrz-business-direct...


In [30]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(df2)
words = cv.get_feature_names()

In [31]:
len(words)

17226

In [32]:
cv2 = CountVectorizer(analyzer='word', ngram_range=(2,2))
X2 = cv2.fit_transform(df2)
pairs = cv2.get_feature_names()

In [33]:
len(pairs)

83221

In [34]:
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer()
X = hv.fit_transform(df2)
print(X.shape)

(4689, 1048576)


In [35]:
def lower_case(df, col):
    df['text'] = df.text.apply(lambda x: " ".join(x.lower() for x in x.split()))
    print(df.text.head())

lower_case(tw, 'text')

0    @jnthn_lckwd @jbarro @corybooker understood. w...
1    in accordance with current public health recom...
2    the american people should know if they're sic...
3    rt @court_rice leadership: @cartwrightpa launc...
4    https://www.wbrz.com/news/wbrz-business-direct...
Name: text, dtype: object


In [36]:
tw.head()

,text
0,@jnthn_lckwd @jbarro @corybooker understood. w...
1,in accordance with current public health recom...
2,the american people should know if they're sic...
3,rt @court_rice leadership: @cartwrightpa launc...
4,https://www.wbrz.com/news/wbrz-business-direct...


In [37]:
tw['callouts'] = tw.text.apply(lambda x: re.findall(r'@[\S]+', str(x)))
tw['callouts'].head()

0    [@jnthn_lckwd, @jbarro, @corybooker]
1                                      []
2     [@esmemurphy, @amyklobuchar, @wcco]
3            [@court_rice, @cartwrightpa]
4                                      []
Name: callouts, dtype: object

In [38]:
tw['hashtags'] = tw.text.apply(lambda x: re.findall(r'#[\S]+', str(x)))
tw.hashtags.head()

0         []
1         []
2         []
3    [#pa08]
4         []
Name: hashtags, dtype: object

In [39]:
charref = [x for x in df.text.apply(lambda x: re.findall(r'&[\S]+', str(x)))]
charref
flat_list = [item for sublist in charref for item in sublist]
len(flat_list)

1306

In [40]:
flat_list[:20]

['&amp;',
 '&amp;',
 '&amp;',
 '&utm_source=twitter.com&utm_content=tw-RecordBee&utm_campaign=socialflow',
 '&utm_source=twitter.com&utm_medium=social&utm_campaign=politics',
 '&amp;',
 '&amp;',
 '&amp;',
 '&amp;',
 '&gt;&gt;',
 '&amp;',
 '&amp;',
 '&amp;',
 '&amp;',
 '&amp;',
 '&amp;',
 '&amp;',
 '&amp;',
 '&amp;',
 '&amp;']

In [42]:
# extract all hashtags into new column
df['hashtags'] = df.text.apply(lambda x: re.findall(r'#\S+', str(x)))
df.hashtags.head()

0         []
1         []
2         []
3    [#PA08]
4         []
Name: hashtags, dtype: object

In [43]:
df['callouts'] = df.text.apply(lambda x: re.findall(r'@\S+', str(x)))
df['callouts'].head()

0    [@JNTHN_LCKWD, @jbarro, @CoryBooker]
1                                      []
2     [@esmemurphy, @amyklobuchar, @wcco]
3            [@Court_Rice, @CartwrightPA]
4                                      []
Name: callouts, dtype: object

In [46]:
charref = [x for x in df.text.apply(lambda x: re.findall(r'&[\S]+', str(x)))]
charref
flat_list = [item for sublist in charref for item in sublist]
len(flat_list)

1306

In [47]:
# replace all hashtags, ampersands, and callouts with no space
df.text = df.text.apply(lambda x: re.sub(r'[#@&][\S]+', '', str(x)))
df.text.head(30)

0        Understood. Working this hard. Don’t know w...
1     In accordance with current public health recom...
2     The American people should know if they're sic...
3     RT  LEADERSHIP:  launched a hotline yesterday ...
4     https://www.wbrz.com/news/wbrz-business-direct...
5     A big update for small businesses and those wh...
6     RT  When it comes to combating  Texans should ...
7                                           And hockey!
8     RT  𝘋𝘪𝘥 𝘺𝘰𝘶 𝘬𝘯𝘰𝘸 𝘵𝘩𝘢𝘵 1 𝘶𝘯𝘪𝘵 𝘰𝘧 𝘥𝘰𝘯𝘢𝘵𝘦𝘥 𝘣𝘭𝘰𝘰𝘥 ...
9     . has suspended their voter contact program an...
10    RT  Thanks to  who shared in our daily call th...
11    With  at the center of the  outbreak, we need ...
12    RT  CURFEW: In  by executive order of  Demings...
13    I'm hosting a  Telephone Town Hall on Monday, ...
14    It’s great, of course. But, you all do realize...
15    RT  While Congress continues its work respondi...
16    RT  Tribes need the resources to fight  \n\nTh...
17    RT  Garamendi applauds signing of Families

In [48]:
# remove twitter RT and QT
df.text = df.text.apply(lambda x: re.sub(r'(\bRT.*|\bQT.*)', '', str(x)))
df.text.head(30)

0        Understood. Working this hard. Don’t know w...
1     In accordance with current public health recom...
2     The American people should know if they're sic...
3     \n\nhttps://www.thetimes-tribune.com/coronavir...
4     https://www.wbrz.com/news/wbrz-business-direct...
5     A big update for small businesses and those wh...
6                                                      
7                                           And hockey!
8     \n \n“During this crisis, we all need to pitch...
9     . has suspended their voter contact program an...
10                              \n\n1-888-407-4747\n\n 
11    With  at the center of the  outbreak, we need ...
12                                                     
13    I'm hosting a  Telephone Town Hall on Monday, ...
14    It’s great, of course. But, you all do realize...
15    \n\n✔️ Buy gift cards\n✔️ Order take out or de...
16    \n\nThey need this money now to prepare and ha...
17                                              

In [49]:
df.text.head(30)

0        Understood. Working this hard. Don’t know w...
1     In accordance with current public health recom...
2     The American people should know if they're sic...
3     \n\nhttps://www.thetimes-tribune.com/coronavir...
4     https://www.wbrz.com/news/wbrz-business-direct...
5     A big update for small businesses and those wh...
6                                                      
7                                           And hockey!
8     \n \n“During this crisis, we all need to pitch...
9     . has suspended their voter contact program an...
10                              \n\n1-888-407-4747\n\n 
11    With  at the center of the  outbreak, we need ...
12                                                     
13    I'm hosting a  Telephone Town Hall on Monday, ...
14    It’s great, of course. But, you all do realize...
15    \n\n✔️ Buy gift cards\n✔️ Order take out or de...
16    \n\nThey need this money now to prepare and ha...
17                                              

In [50]:
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

In [51]:
# remove html links
df.text = df.text.apply(lambda x:re.sub(r'(http)?', '', str(x)))
df.text.head(30)

0        Understood. Working this hard. Don’t know w...
1     In accordance with current public health recom...
2     The American people should know if they're sic...
3     \n\ns://www.thetimes-tribune.com/coronavirus/c...
4     s://www.wbrz.com/news/wbrz-business-directory-...
5     A big update for small businesses and those wh...
6                                                      
7                                           And hockey!
8     \n \n“During this crisis, we all need to pitch...
9     . has suspended their voter contact program an...
10                              \n\n1-888-407-4747\n\n 
11    With  at the center of the  outbreak, we need ...
12                                                     
13    I'm hosting a  Telephone Town Hall on Monday, ...
14    It’s great, of course. But, you all do realize...
15    \n\n✔️ Buy gift cards\n✔️ Order take out or de...
16    \n\nThey need this money now to prepare and ha...
17                                              

In [52]:
def expand_contractions(text):
    text = contractions.fix(text)
    return text

df.text = df.text.apply(lambda x: expand_contractions(x))

AttributeError: module 'contractions' has no attribute 'fix'

In [53]:
df.text = df.text.apply(lambda x: re.sub(r'[^\w\s]', r'', str(x)))
df.text.head(30)

0        Understood Working this hard Dont know what...
1     In accordance with current public health recom...
2     The American people should know if theyre sick...
3     \n\nswwwthetimestribunecomcoronaviruscartwrigh...
4     swwwwbrzcomnewswbrzbusinessdirectorystorehours...
5     A big update for small businesses and those wh...
6                                                      
7                                            And hockey
8     \n \nDuring this crisis we all need to pitch i...
9      has suspended their voter contact program and...
10                                 \n\n18884074747\n\n 
11    With  at the center of the  outbreak we need t...
12                                                     
13    Im hosting a  Telephone Town Hall on Monday Ma...
14    Its great of course But you all do realize tha...
15    \n\n Buy gift cards\n Order take out or delive...
16    \n\nThey need this money now to prepare and ha...
17                                              

In [54]:
# lower casing
df.text = df.text.apply(lambda x: " ".join(x.lower() for x in x.split()))
df.text.head(30)

0     understood working this hard dont know what th...
1     in accordance with current public health recom...
2     the american people should know if theyre sick...
3     swwwthetimestribunecomcoronaviruscartwrightpre...
4     swwwwbrzcomnewswbrzbusinessdirectorystorehours...
5     a big update for small businesses and those wh...
6                                                      
7                                            and hockey
8     during this crisis we all need to pitch in ple...
9     has suspended their voter contact program and ...
10                                          18884074747
11    with at the center of the outbreak we need to ...
12                                                     
13    im hosting a telephone town hall on monday mar...
14    its great of course but you all do realize tha...
15    buy gift cards order take out or delivery prom...
16    they need this money now to prepare and have t...
17                                              

In [55]:
text = """three cups of coffee"""
doc = nlp(text)
tokens = [w2n.word_to_num(token.text) if token.pos_ == 'NUM' else token for token in doc]

print(tokens) # result: [3, cups, of, coffee]


[3, cups, of, coffee]


In [56]:
# remove numbers or words with digits  
df.text = df.text.apply(lambda x: re.sub(r'\w*\d\w*', r'', str(x)))
df.text.head(30)

0     understood working this hard dont know what th...
1     in accordance with current public health recom...
2     the american people should know if theyre sick...
3                                                      
4     swwwwbrzcomnewswbrzbusinessdirectorystorehours...
5     a big update for small businesses and those wh...
6                                                      
7                                            and hockey
8     during this crisis we all need to pitch in ple...
9     has suspended their voter contact program and ...
10                                                     
11    with at the center of the outbreak we need to ...
12                                                     
13    im hosting a telephone town hall on monday mar...
14    its great of course but you all do realize tha...
15    buy gift cards order take out or delivery prom...
16    they need this money now to prepare and have t...
17                                              

In [57]:
# remove any double or more whitespaces to single whitespaces
df.text = df.text.apply(lambda x: re.sub(r'\s\s+', ' ', str(x)))
df.text.head(30)

0     understood working this hard dont know what th...
1     in accordance with current public health recom...
2     the american people should know if theyre sick...
3                                                      
4     swwwwbrzcomnewswbrzbusinessdirectorystorehours...
5     a big update for small businesses and those wh...
6                                                      
7                                            and hockey
8     during this crisis we all need to pitch in ple...
9     has suspended their voter contact program and ...
10                                                     
11    with at the center of the outbreak we need to ...
12                                                     
13    im hosting a telephone town hall on monday mar...
14    its great of course but you all do realize tha...
15    buy gift cards order take out or delivery prom...
16    they need this money now to prepare and have t...
17                                              

In [58]:
# remove any leading and trailing whitespace
df.text = df.text.apply(lambda x: re.sub(r'(\A\s+|\s+\Z)', '', str(x)))
df.text.head(30)

0     understood working this hard dont know what th...
1     in accordance with current public health recom...
2     the american people should know if theyre sick...
3                                                      
4     swwwwbrzcomnewswbrzbusinessdirectorystorehours...
5     a big update for small businesses and those wh...
6                                                      
7                                            and hockey
8     during this crisis we all need to pitch in ple...
9     has suspended their voter contact program and ...
10                                                     
11    with at the center of the outbreak we need to ...
12                                                     
13    im hosting a telephone town hall on monday mar...
14    its great of course but you all do realize tha...
15    buy gift cards order take out or delivery prom...
16    they need this money now to prepare and have t...
17                                              

In [59]:
import nltk
from nltk.corpus import gutenberg, stopwords
from nltk.collocations import *
from nltk import FreqDist
from nltk import word_tokenize
import string
import re

In [60]:
f = open('data/austen-emma.txt')
raw = f.read()
print(len(raw))
raw[:75]

887071


'[Emma by Jane Austen 1816]\n\nVOLUME I\n\nCHAPTER I\n\n\nEmma Woodhouse, handsome,'

In [61]:
tokens = word_tokenize(raw)
print(type(tokens))
print(len(tokens))
tokens[:10]

<class 'list'>
191785


['[', 'Emma', 'by', 'Jane', 'Austen', '1816', ']', 'VOLUME', 'I', 'CHAPTER']

In [62]:
text = nltk.Text(tokens)
print(type(text))
raw.find("VOLUME I")

<class 'nltk.text.Text'>


28

In [63]:
words = [w.lower() for w in tokens]
print(type(words))
words[1:25]

<class 'list'>


['emma',
 'by',
 'jane',
 'austen',
 '1816',
 ']',
 'volume',
 'i',
 'chapter',
 'i',
 'emma',
 'woodhouse',
 ',',
 'handsome',
 ',',
 'clever',
 ',',
 'and',
 'rich',
 ',',
 'with',
 'a',
 'comfortable',
 'home']

In [64]:
fdist1 = FreqDist(words)
fdist1.most_common(20)

[(',', 12016),
 ('.', 6355),
 ('the', 5201),
 ('to', 5181),
 ('and', 4877),
 ('of', 4284),
 ('i', 3177),
 ('a', 3124),
 ('--', 3100),
 ('it', 2503),
 ("''", 2452),
 ('her', 2448),
 ('was', 2396),
 (';', 2353),
 ('she', 2336),
 ('not', 2281),
 ('in', 2173),
 ('be', 1970),
 ('you', 1967),
 ('he', 1806)]

In [65]:
V = set(words)
long_words = [w for w in V if len(w) > 15]
sorted(long_words)

['backgammon-table',
 'cheerful-tempered',
 'companionableness',
 'congratulations.',
 'conscience-stricken',
 'disagreeableness',
 'disingenuousness',
 'disinterestedness',
 'eight-and-thirty',
 'fore-shortening.',
 'gentleman-farmer',
 'incomprehensible',
 'manchester-street',
 'misunderstanding',
 'misunderstandings',
 'open-heartedness',
 'self-approbation',
 'self-consequence',
 'self-contentment',
 'self-examination',
 'seven-and-twenty',
 'strawberry-beds.',
 'three-and-twenty',
 'unceremoniousness',
 'undistinguishing',
 'unseasonableness',
 'wedding-present.']

In [66]:
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
tokens_raw = nltk.regexp_tokenize(raw, pattern)
tokens = [word.lower() for word in tokens_raw]

In [67]:
fd = FreqDist(tokens)
fd.most_common(10)

[('to', 5242),
 ('the', 5204),
 ('and', 4897),
 ('of', 4293),
 ('i', 3191),
 ('a', 3130),
 ('it', 2529),
 ('her', 2483),
 ('was', 2400),
 ('she', 2364)]

In [68]:
stopwords_list = stopwords.words('english')
stopwords_list += list(string.punctuation)
stopwords_list += ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

words_stopped = [word for word in tokens if word not in stopwords_list]

In [69]:
stopped_fd = FreqDist(words_stopped)
stopped_fd.most_common(10)

[('mr', 1154),
 ('could', 837),
 ('would', 821),
 ('emma', 786),
 ('mrs', 701),
 ('miss', 602),
 ('must', 571),
 ('much', 486),
 ('said', 484),
 ('one', 448)]

In [70]:
total_count = sum(stopped_fd.values())
top_50 = stopped_fd.most_common(50)
print('Word\t\t\tNormalized Frequency')
for word in top_50:
    norm_freq = word[1] / total_count
    print('{} \t\t\t {:.4}'.format(word[0], norm_freq))

Word			Normalized Frequency
mr 			 0.01574
could 			 0.01141
would 			 0.01119
emma 			 0.01072
mrs 			 0.009559
miss 			 0.008209
must 			 0.007786
much 			 0.006627
said 			 0.0066
one 			 0.006109
every 			 0.005932
harriet 			 0.005659
well 			 0.005495
thing 			 0.005427
weston 			 0.005304
think 			 0.005236
little 			 0.004922
never 			 0.004882
good 			 0.004882
knightley 			 0.004854
know 			 0.004595
might 			 0.004445
elton 			 0.004363
say 			 0.004254
jane 			 0.003845
quite 			 0.003845
time 			 0.003818
woodhouse 			 0.003791
great 			 0.003613
nothing 			 0.003491
dear 			 0.003286
always 			 0.003245
thought 			 0.003082
soon 			 0.003054
may 			 0.003041
see 			 0.003027
man 			 0.002986
shall 			 0.002986
without 			 0.002918
first 			 0.002877
fairfax 			 0.002863
sure 			 0.002782
indeed 			 0.002754
like 			 0.002727
frank 			 0.002727
made 			 0.002714
ever 			 0.002632
oh 			 0.002632
churchill 			 0.002632
young 			 0.002618


In [71]:
bm = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(words_stopped)
scored = finder.score_ngrams(bm.raw_freq)
scored[:20]

[(('mr', 'knightley'), 0.003667998418260905),
 (('mrs', 'weston'), 0.003122571144170064),
 (('mr', 'elton'), 0.0025907795519314945),
 (('miss', 'woodhouse'), 0.0022226161419201767),
 (('mr', 'weston'), 0.0019499025048747563),
 (('frank', 'churchill'), 0.001799910004499775),
 (('every', 'thing'), 0.0017044602315338779),
 (('mrs', 'elton'), 0.001690824549681607),
 (('mr', 'woodhouse'), 0.0014999250037498126),
 (('miss', 'fairfax'), 0.0014862893218975414),
 (('miss', 'bates'), 0.0014044752307839153),
 (('jane', 'fairfax'), 0.0013090254578180182),
 (('every', 'body'), 0.001227211366704392),
 (('young', 'man'), 0.0010908545481816817),
 (('said', 'emma'), 0.0008863193203976165),
 (('great', 'deal'), 0.0008726836385453455),
 (('emma', 'could'), 0.0008454122748408035),
 (('john', 'knightley'), 0.0007635981837271772),
 (('dare', 'say'), 0.0006954197744658222),
 (('miss', 'smith'), 0.0006954197744658222)]

In [72]:
pmi_finder = BigramCollocationFinder.from_words(words_stopped)
pmi_finder.apply_freq_filter(5)
pmi_scored = pmi_finder.score_ngrams(bm.pmi)
pmi_scored[:20]

[(('barouche', 'landau'), 13.354898708356725),
 (('court', 'plaister'), 12.70282201177703),
 (('brunswick', 'square'), 12.577291129693169),
 (('sore', 'throat'), 12.384646051750776),
 (('mill', 'farm'), 11.854825105222083),
 (('william', 'larkins'), 11.769936207635563),
 (('baked', 'apples'), 11.67040053408465),
 (('post', 'office'), 11.461813912273234),
 (('box', 'hill'), 11.349185057162329),
 (('sixteen', 'miles'), 11.22679388260904),
 (('maple', 'grove'), 11.208057320027452),
 (('abbey', 'mill'), 11.20805732002745),
 (('colonel', "campbell's"), 10.721681039028347),
 (('hair', 'cut'), 10.676826803244088),
 (('south', 'end'), 10.57729112969317),
 (('shook', 'head'), 10.42528803624812),
 (('colonel', 'campbell'), 9.880378785047402),
 (('robert', 'martin'), 9.747216131135483),
 (('self', 'command'), 9.501188150607382),
 (('vast', 'deal'), 9.375657268523524)]